# Image Preprocessing
------

In [58]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rishikesh4089/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [59]:
import nltk
nltk.data.path.append('nltk_data')
from nltk.tokenize import word_tokenize
print(word_tokenize("Testing NLTK tokenization now!"))


['Testing', 'NLTK', 'tokenization', 'now', '!']


<a id='step1'></a>
## Step 1: Explore the Data Loader

In [ ]:
import sys
sys.path.append('/cocoapi/PythonAPI')
from pycocotools.coco import COCO
import nltk
nltk.data.path.append('nltk_data')
from captioning.data_loader import get_loader
from torchvision import transforms

transform_train = transforms.Compose([ 
    transforms.Resize(256),                          
    transforms.RandomCrop(224),                      
    transforms.RandomHorizontalFlip(),               
    transforms.ToTensor(),                           
    transforms.Normalize((0.485, 0.456, 0.406),      
                         (0.229, 0.224, 0.225))])

vocab_threshold = 5
batch_size = 10
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=False)

loading annotations into memory...
Done (t=0.34s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:10<00:00, 41340.11it/s]


In [4]:
sample_caption = 'A person doing a trick on a rail while riding a skateboard.'

In [61]:
import nltk

sample_tokens = nltk.tokenize.word_tokenize(str(sample_caption).lower())
print(sample_tokens)

['a', 'person', 'doing', 'a', 'trick', 'on', 'a', 'rail', 'while', 'riding', 'a', 'skateboard', '.']


In [62]:
sample_caption = []

start_word = data_loader.dataset.vocab.start_word
print('Special start word:', start_word)
sample_caption.append(data_loader.dataset.vocab(start_word))
print(sample_caption)

Special start word: <start>
[0]


In [63]:
sample_caption.extend([data_loader.dataset.vocab(token) for token in sample_tokens])
print(sample_caption)

[0, 3, 98, 754, 3, 396, 39, 3, 1010, 207, 139, 3, 753, 18]


In [64]:
end_word = data_loader.dataset.vocab.end_word
print('Special end word:', end_word)

sample_caption.append(data_loader.dataset.vocab(end_word))
print(sample_caption)

Special end word: <end>
[0, 3, 98, 754, 3, 396, 39, 3, 1010, 207, 139, 3, 753, 18, 1]


In [65]:
import torch

sample_caption = torch.Tensor(sample_caption).long()
print(sample_caption)

tensor([   0,    3,   98,  754,    3,  396,   39,    3, 1010,  207,  139,    3,
         753,   18,    1])


In [66]:
dict(list(data_loader.dataset.vocab.word2idx.items())[:10])

{'<start>': 0,
 '<end>': 1,
 '<unk>': 2,
 'a': 3,
 'very': 4,
 'clean': 5,
 'and': 6,
 'well': 7,
 'decorated': 8,
 'empty': 9}

We also print the total number of keys.

In [67]:
print('Total number of tokens in vocabulary:', len(data_loader.dataset.vocab))

Total number of tokens in vocabulary: 8852


In [68]:
vocab_threshold = 4
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=False)

loading annotations into memory...
Done (t=0.30s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:09<00:00, 41826.32it/s]


In [69]:
print('Total number of tokens in vocabulary:', len(data_loader.dataset.vocab))

Total number of tokens in vocabulary: 9947


In [70]:
unk_word = data_loader.dataset.vocab.unk_word
print('Special unknown word:', unk_word)

print('All unknown words are mapped to this integer:', data_loader.dataset.vocab(unk_word))

Special unknown word: <unk>
All unknown words are mapped to this integer: 2


Check this for yourself below, by pre-processing the provided nonsense words that never appear in the training captions. 

In [71]:
print(data_loader.dataset.vocab('jfkafejw'))
print(data_loader.dataset.vocab('ieowoqjf'))

2
2


In [72]:
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_from_file=True)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.31s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:09<00:00, 42891.47it/s]


In the next section, you will learn how to use the data loader to obtain batches of training data.

<a id='step2'></a>
## Step 2: Use the Data Loader function to obtain Batches

In [74]:
from collections import Counter

counter = Counter(data_loader.dataset.caption_lengths)
lengths = sorted(counter.items(), key=lambda pair: pair[1], reverse=True)
for value, count in lengths:
    print('value: %2d --- count: %5d' % (value, count))

value: 10 --- count: 86302
value: 11 --- count: 79971
value:  9 --- count: 71920
value: 12 --- count: 57653
value: 13 --- count: 37668
value: 14 --- count: 22342
value:  8 --- count: 20742
value: 15 --- count: 12839
value: 16 --- count:  7736
value: 17 --- count:  4845
value: 18 --- count:  3101
value: 19 --- count:  2017
value:  7 --- count:  1594
value: 20 --- count:  1453
value: 21 --- count:   997
value: 22 --- count:   683
value: 23 --- count:   534
value: 24 --- count:   384
value: 25 --- count:   277
value: 26 --- count:   214
value: 27 --- count:   160
value: 28 --- count:   114
value: 29 --- count:    87
value: 30 --- count:    58
value: 31 --- count:    49
value: 32 --- count:    44
value: 34 --- count:    40
value: 37 --- count:    32
value: 35 --- count:    31
value: 33 --- count:    30
value: 36 --- count:    26
value: 38 --- count:    18
value: 39 --- count:    18
value: 43 --- count:    16
value: 44 --- count:    16
value: 48 --- count:    12
value: 45 --- count:    11
v

In [76]:
import numpy as np
import torch.utils.data as data

indices = data_loader.dataset.get_train_indices()
print('sampled indices:', indices)
new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
data_loader.batch_sampler.sampler = new_sampler
images, captions = next(iter(data_loader))
print('images.shape:', images.shape)
print('captions.shape:', captions.shape)

sampled indices: [np.int64(72408), np.int64(243954), np.int64(401547), np.int64(404681), np.int64(156783), np.int64(222167), np.int64(368440), np.int64(241526), np.int64(10976), np.int64(90693)]
images.shape: torch.Size([10, 3, 224, 224])
captions.shape: torch.Size([10, 13])


<a id='step3'></a>
## Step 3: Import Encoder and Decoder CNN

In [ ]:
from captioning.model import EncoderCNN, DecoderRNN

In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [80]:
embed_size = 256
encoder = EncoderCNN(embed_size)
encoder.to(device)
images = images.to(device)
features = encoder(images)
print('type(features):', type(features))
print('features.shape:', features.shape)

assert type(features)==torch.Tensor, "Encoder output needs to be a PyTorch Tensor." 
assert (features.shape[0]==batch_size) & (features.shape[1]==embed_size), "The shape of the encoder output is incorrect."

/Users/rishikesh4089/Desktop/Work/ZIDIO_Task1/venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/rishikesh4089/Desktop/Work/ZIDIO_Task1/venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/rishikesh4089/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:53<00:00, 1.90MB/s]


type(features): <class 'torch.Tensor'>
features.shape: torch.Size([10, 256])


<a id='step4'></a>
## Step 4: Implement the RNN Decoder

In [81]:
hidden_size = 512
vocab_size = len(data_loader.dataset.vocab)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)
decoder.to(device)
captions = captions.to(device)
outputs = decoder(features, captions)
print('type(outputs):', type(outputs))
print('outputs.shape:', outputs.shape)

assert type(outputs)==torch.Tensor, "Decoder output needs to be a PyTorch Tensor."
assert (outputs.shape[0]==batch_size) & (outputs.shape[1]==captions.shape[1]) & (outputs.shape[2]==vocab_size), "The shape of the decoder output is incorrect."

type(outputs): <class 'torch.Tensor'>
outputs.shape: torch.Size([10, 13, 9947])


In [ ]:
import torch

print(f"torch.backends.mps.is_available(): {torch.backends.mps.is_available()}")
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using Apple Silicon MPS (Metal Performance Shaders) for GPU acceleration.")
elif torch.cuda.is_available(): 
    device = torch.device("cuda")
    print("Using NVIDIA CUDA for GPU acceleration.")
else:
    device = torch.device("cpu")
    print("Using CPU for training.")

torch.backends.mps.is_available(): True
Using Apple Silicon MPS (Metal Performance Shaders) for GPU acceleration.
